In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
#Word stemmer
stemmer = LancasterStemmer()


In [2]:
# Step2
# provide 3 classes of training data
training_data=[]
training_data.append({"class":"greeting", "sentence":"how are you?"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})
#
training_data.append({"class":"goodbye", "sentence":"have a nice day"})
training_data.append({"class":"goodbye", "sentence":"see you later"})
training_data.append({"class":"goodbye", "sentence":"talk to you soon"})
#
training_data.append({"class":"sandwich", "sentence":"make me a sandwich"})
training_data.append({"class":"sandwich", "sentence":"can you make a sandwich?"})
training_data.append({"class":"sandwich", "sentence":"having a sandwich today?"})
training_data.append({"class":"sandwich", "sentence":"what's for lunch?"})
print ("%s sentences of training data" % len(training_data))

#training_data.append({"class":"Aegis", "sentence":"what is aegis?"})
#training_data.append({"class":"Aegis", "sentence":"where is aegis located?"})
#training_data.append({"class":"Aegis", "sentence":"what is admission process in aegis"})
#training_data.append({"class":"Aegis", "sentence":"Do aegis provide placements?"})
#
#print "%s sentences of training data" %len(training_data)



11 sentences of training data


In [3]:
responses_data =[]
#
responses_data.append({"class":"greeting", "sentence":"I am fine"})
responses_data.append({"class":"greeting", "sentence":"Good what about you"})
responses_data.append({"class":"greeting", "sentence":"Welcome"})
responses_data.append({"class":"greeting", "sentence":"Going good thank you."})
#
responses_data.append({"class":"goodbye", "sentence":"thank you wish you the same."})
responses_data.append({"class":"goodbye", "sentence":"okay cheers."})
responses_data.append({"class":"goodbye", "sentence":"pleasure is mine"})
#
responses_data.append({"class":"Aegis", "sentence":"Data science school"})
responses_data.append({"class":"Aegis", "sentence":"Powai mumbai"})
responses_data.append({"class":"Aegis", "sentence":"you have to pay 1000rs as admission confirmation"})
responses_data.append({"class":"Aegis", "sentence":"yes Aegis provide placements"})
#
print "%s sentences for response " %len(responses_data)


11 sentences for response 


In [4]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

Corpus words and counts: {'is': 2, 'it': 1, 'see': 1, 'ar': 1, 'what': 1, 'for': 1, 'how': 3, 'to': 1, 'going': 1, 'you': 4, 'hav': 2, 'can': 1, 'me': 1, 'good': 1, 'yo': 1, 'nic': 1, 'soon': 1, 'lunch': 1, 'lat': 1, 'mak': 2, 'day': 3, 'a': 4, 'sandwich': 3, 'today': 2, 'talk': 1} 

Class words: {'sandwich': ['mak', 'me', 'a', 'sandwich', 'can', 'you', 'mak', 'a', 'sandwich', 'hav', 'a', 'sandwich', 'today', 'what', 'for', 'lunch'], 'greeting': ['how', 'ar', 'you', 'how', 'is', 'yo', 'day', 'good', 'day', 'how', 'is', 'it', 'going', 'today'], 'goodbye': ['hav', 'a', 'nic', 'day', 'see', 'you', 'lat', 'talk', 'to', 'you', 'soon']}


In [5]:
# we can now calculate a score for a new sentence
sentence = "good day for us to have lunch?"

# calculate a score for a given class
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with same weight
            score += 1
            
            if show_details:
                print ("   match: %s" % stemmer.stem(word.lower() ))
    return score


In [6]:
# now we can find the class with the highest score
for c in class_words.keys():
    print "Class: %s  Score: %s \n" %(c, calculate_class_score(sentence, c))

   match: for
   match: hav
   match: lunch
Class: sandwich  Score: 3 

   match: good
   match: day
Class: greeting  Score: 2 

   match: day
   match: to
   match: hav
Class: goodbye  Score: 3 



In [9]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score_commonality(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [10]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score_commonality(sentence, c)))

   match: for (1)
   match: hav (0)
   match: lunch (1)
Class: sandwich  Score: 2 

   match: good (1)
   match: day (0)
Class: greeting  Score: 1 

   match: day (0)
   match: to (1)
   match: hav (0)
Class: goodbye  Score: 1 



In [11]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score_commonality(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [12]:
classify("how are you doing today?")

('greeting', 1)